<a href="https://colab.research.google.com/github/cjvilla/enose_predictions/blob/main/exploratory_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [17]:
sensor_data = pd.read_csv('../content/drive/MyDrive/eNose/sensor_data.csv')

In [18]:
sensor_data.columns

Index(['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11',
       'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21',
       'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31',
       'S32', 'real_idn', 'group'],
      dtype='object')

In [26]:
sensor_data.count()

S1          6160
S2          6160
S3          6160
S4          6160
S5          6160
S6          6160
S7          6160
S8          6160
S9          6160
S10         6160
S11         6160
S12         6160
S13         6160
S14         6160
S15         6160
S16         6160
S17         6160
S18         6160
S19         6160
S20         6160
S21         6160
S22         6160
S23         6160
S24         6160
S25         6160
S26         6160
S27         6160
S28         6160
S29         6160
S30         6160
S31         6160
S32         6160
real_idn    6160
group       6160
dtype: int64

In [14]:
demographics = pd.read_csv('/content/drive/MyDrive/eNose/demographics.csv')

In [27]:
demographics.count()

real_idn          60
Age               60
Sex               60
Race              60
BMI               60
LPS_Endotoxin     60
Perm_sucralose    60
S1_mean           60
S2_mean           60
S3_mean           60
S4_mean           60
S5_mean           60
S6_mean           60
S7_mean           60
S8_mean           60
S9_mean           60
S10_mean          60
S11_mean          60
S12_mean          60
S13_mean          60
S14_mean          60
S15_mean          60
S16_mean          60
S17_mean          60
S18_mean          60
S19_mean          60
S20_mean          60
S21_mean          60
S22_mean          60
S23_mean          60
S24_mean          60
S25_mean          60
S26_mean          60
S27_mean          60
S28_mean          60
S29_mean          60
S30_mean          60
S31_mean          60
S32_mean          60
group             60
dtype: int64

In [19]:
demographics.head()

,real_idn,Age,Sex,Race,BMI,LPS_Endotoxin,Perm_sucralose,S1_mean,S2_mean,S3_mean,...,S24_mean,S25_mean,S26_mean,S27_mean,S28_mean,S29_mean,S30_mean,S31_mean,S32_mean,group
0,ALD018,63,1,4,0.00,1.560,2.934,3.841947,4.682487,1.481698,...,6.279635,1.837717,2.624174,0.883194,4.253827,1.893487,2.121259,1.650041,1.086850,1
1,ALD081,38,2,4,0.00,3.010,2.356,3.839645,4.680415,1.481503,...,6.277149,1.837357,2.620586,0.882853,4.253657,1.891885,2.121258,1.656272,1.086544,1
2,ALD094,42,2,4,0.00,0.273,2.633,3.841267,4.681959,1.481513,...,6.279084,1.837666,2.623186,0.883147,4.253923,1.893046,2.121164,1.656858,1.086777,1
3,ALD095,41,1,4,0.00,2.580,2.210,3.842414,4.683155,1.481880,...,6.278215,1.837813,2.624733,0.883220,4.253709,1.893847,2.121325,1.652564,1.086866,1
4,ALD096,48,2,0,21.52,1.450,2.320,3.842594,4.683291,1.481914,...,6.279254,1.837857,2.625273,0.883282,4.254359,1.894120,2.121380,1.659235,1.086897,1


In [22]:
merged_df = pd.merge(sensor_data, demographics, on=['real_idn', 'group'], how='inner')


In [29]:
merged_df.head(True)

TypeError: cannot do positional indexing on Int64Index with these indexers [True] of type bool

In [25]:
merged_df.count()

S1          6160
S2          6160
S3          6160
S4          6160
S5          6160
            ... 
S28_mean    6160
S29_mean    6160
S30_mean    6160
S31_mean    6160
S32_mean    6160
Length: 72, dtype: int64